In [ ]:
# Import py2neo to work with Neo4j Graph Database

In [64]:
import pandas as pd
from py2neo import Graph, Node, Relationship
from graphdatascience import GraphDataScience

# Import Dataset and Data cleaning 

In [65]:
fare_df = pd.read_csv('fare_rider_categories_bart.csv')
time_df = pd.read_csv('bart_time.csv')

In [66]:
fare_df

,fare_id,rider_category_id,price,origin_id,destination_id,origin,destination,origin_station,destination_station,rider_category_name
0,1,5,1.80,0,1,SSAN,SBRN,South San Francisco,San Bruno,Youth Clipper
1,2,5,3.20,0,0,SSAN,SSAN,South San Francisco,South San Francisco,Youth Clipper
2,3,5,4.25,0,2,SSAN,SFIA,South San Francisco,San Francisco International Airport,Youth Clipper
3,4,5,1.80,0,3,SSAN,MLBR,South San Francisco,Millbrae,Youth Clipper
4,5,5,3.65,0,4,SSAN,WDUB,South San Francisco,West Dublin/Pleasanton,Youth Clipper
...,...,...,...,...,...,...,...,...,...,...
9995,2496,1,9.20,49,45,ANTC,WARM,Antioch,Warm Springs/South Fremont,Clipper
9996,2497,1,10.00,49,46,ANTC,MLPT,Antioch,Milpitas,Clipper
9997,2498,1,10.30,49,47,ANTC,BERY,Antioch,Berryessa/North San Jose,Clipper
9998,2499,1,2.15,49,48,ANTC,PCTR,Antioch,Pittsburg Center,Clipper


In [67]:
time_df = time_df.drop_duplicates()
time_df.to_csv("bart_time.csv", index=False)

In [68]:
time_df.head()

,origin_id,destination_id,time
0,13,12,2
1,13,14,3
2,39,38,2
3,12,11,4
4,12,13,2


# Connect to Neo4j Database

In [69]:
graph_name = "neo4j" 
SERVER_ADDRESS = "bolt://localhost:7687"
SERVER_AUTH = ("neo4j", "root@420")
graph = Graph(SERVER_ADDRESS, auth=SERVER_AUTH)
print("Conection established with Neo4j")

Conection established with Neo4j


# Create the graph with stations as nodes and fare as relationship between stations 

In [70]:
def create_stations_fares(fare_df, graph):
    for index, row in fare_df.iterrows():
        origin_id = row['origin_id']
        origin_node = graph.nodes.match("Station", station_id=origin_id).first()
        if origin_node:
            origin = origin_node
        else:
            origin = Node("Station", station_id=origin_id, station_name=row['origin_station'])

        destination_id = row['destination_id']
        destination_node = graph.nodes.match("Station", station_id=destination_id).first()
        if destination_node:
            destination = destination_node
        else:
            destination = Node("Station", station_id=destination_id, station_name=row['destination_station'])

        # check if a fare relationship already exists between the source and destination nodes
        query = f"MATCH (o:Station {{station_id: '{origin_id}'}})-[f:fare]-(:Station {{station_id: '{destination_id}'}}) RETURN f"
        result = graph.run(query).data()
        if result:
            existing_fare = result[0]['f']

            if row['rider_category_id'] == 5:
                existing_fare['youth_fare'] = row['price']
            elif row['rider_category_id'] == 2:
                existing_fare['senior_fare'] = row['price']
            elif row['rider_category_id'] == 16:
                existing_fare['clipper_start_fare'] = row['price']
            elif row['rider_category_id'] == 1:
                existing_fare['clipper_fare'] = row['price']

            graph.push(existing_fare)
        else:
            fare = Relationship(origin, "fare", destination,
                                youth_fare=row['price'] if row['rider_category_id'] == 5 else None,
                                senior_fare=row['price'] if row['rider_category_id'] == 2 else None,
                                clipper_start_fare=row['price'] if row['rider_category_id'] == 16 else None,
                                clipper_fare=row['price'] if row['rider_category_id'] == 1 else None)

        # create same relationship from destination to source
        query = f"MATCH (o:Station {{station_id: '{destination_id}'}})-[f:fare]-(:Station {{station_id: '{origin_id}'}}) RETURN f"
        result = graph.run(query).data()
        if result:
            existing_fare = result[0]['f']

            if row['rider_category_id'] == 5:
                existing_fare['youth_fare'] = row['price']
            elif row['rider_category_id'] == 2:
                existing_fare['senior_fare'] = row['price']
            elif row['rider_category_id'] == 16:
                existing_fare['clipper_start_fare'] = row['price']
            elif row['rider_category_id'] == 1:
                existing_fare['clipper_fare'] = row['price']

            graph.push(existing_fare)
        else:
            graph.create(fare)
            fare = Relationship(destination, "fare", origin,
                                youth_fare=row['price'] if row['rider_category_id'] == 5 else None,
                                senior_fare=row['price'] if row['rider_category_id'] == 2 else None,
                                clipper_start_fare=row['price'] if row['rider_category_id'] == 16 else None,
                                clipper_fare=row['price'] if row['rider_category_id'] == 1 else None)        
            graph.create(fare)

    print("Station Nodes and Fare relationships created successfully")


# Create travel_time relationship between stations

In [71]:
def create_stations_time(time_df, graph):
    for index, row in time_df.iterrows():
        origin_id = int(row['origin_id'])  # Convert to Python int
        origin = graph.nodes.match("Station", station_id=origin_id).first()

        destination_id = int(row['destination_id'])  # Convert to Python int
        destination = graph.nodes.match("Station", station_id=destination_id).first()

        # create time relationship from source to destination
        time = Relationship(origin, "travel_time", destination, time=int(row['time']))
        graph.create(time)

        # create the same relationship from destination to source
        time = Relationship(destination, "travel_time", origin, time=int(row['time']))
        graph.create(time)

    print("Time relationships created successfully")


# Create Nodes and Realationships 

In [72]:
create_stations_fares(fare_df, graph)

Station Nodes and Fare relationships created successfully


In [73]:
create_stations_time(time_df, graph)

Time relationships created successfully
